In [1]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [2]:
import tensorflow as tf
from modules.models import Model
from modules.utils import get_dataset
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

2023-02-24 08:30:22.027830: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
ROOT_PATH = "/home/jovyan/ChestXray-14"
INPUT_PATH = f"{ROOT_PATH}/dataset/ChestXray NIH"
LABELS = ['No Finding']

In [4]:
class Dataset:
    INPUT_PATH = INPUT_PATH
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/binary_under_sampling/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/binary_under_sampling/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

## Model Training

In [5]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/experiments/BinaryModelWithUnderSampling'

In [6]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [7]:
def get_callbacks(NAME, weight_option, fold_num=None):
    model_checkpoint_callback = ModelCheckpoint(f'results/models/CrossEntropy/{NAME}_{weight_option}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback

In [8]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100

In [9]:
weight_option = None # use `imagenet` or `None` only

In [11]:
# Callbacks
model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback = get_callbacks(NAME, weight_option)

# Path for CSV
path = os.path.join(CURRENT_PATH, "results", "history", "training_with_CrossEntroy", f"{NAME}_{weight_option}")
os.makedirs(path, exist_ok=True)

# CSV Logger
csv_logger = CSVLogger(os.path.join(path, f"history.csv"))

# Dataset
train_dataset, test_dataset = Dataset().get_train(), Dataset().get_test()

# Modeling
transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False, 
    weights=weight_option,
    input_shape=(224, 224, 3),
    pooling=None
)

model = Model(
    transfer_model,
)
model = model.get_model()
model._name = "BinaryModelCrossEntropyOnSamplingDataset"
model.summary()

Model: "BinaryModelCrossEntropyOnSamplingDataset"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 128)               8028288   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)          

In [12]:
# Visualize
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    verbose=1, # Show Progress Bar while Traning
    callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
)

Epoch 1/100


2023-02-24 08:31:01.612407: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-24 08:31:02.868665: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-24 08:31:02.870478: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-24 08:31:02.870508: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-24 08:31:02.871422: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-24 08:31:02.871488: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-02-24 08:31:07.124686: I tensorflow/stream_executor/cuda/c

3060/3060 [==============================] - 668s 212ms/step - loss: 0.4430 - auc_1: 0.5734 - val_loss: 0.4364 - val_auc_1: 0.5412 - lr: 0.0010
Epoch 2/100
3060/3060 [==============================] - 640s 209ms/step - loss: 0.4231 - auc_1: 0.6133 - val_loss: 0.4144 - val_auc_1: 0.6566 - lr: 0.0010
Epoch 3/100
3060/3060 [==============================] - 642s 210ms/step - loss: 0.4163 - auc_1: 0.6349 - val_loss: 0.4097 - val_auc_1: 0.6651 - lr: 0.0010
Epoch 4/100
3060/3060 [==============================] - 637s 208ms/step - loss: 0.4110 - auc_1: 0.6564 - val_loss: 0.4067 - val_auc_1: 0.6739 - lr: 0.0010
Epoch 5/100
3060/3060 [==============================] - 642s 210ms/step - loss: 0.4117 - auc_1: 0.6535 - val_loss: 0.4094 - val_auc_1: 0.6625 - lr: 0.0010
Epoch 6/100
3060/3060 [==============================] - 633s 207ms/step - loss: 0.4106 - auc_1: 0.6559 - val_loss: 0.4003 - val_auc_1: 0.6926 - lr: 0.0010
Epoch 7/100
3060/3060 [==============================] - 633s 207ms/step - l